Segmentación del mercado de adolecentes en SparkR
===

* *30 min* | Última modificación: Junio 22, 2019

En este tutorial se aplica el algoritmo K-means para clasificar un grupo de adolecentes con base en sus intéreses, con el fin de diseñar estrategias publicitarias y servicios encaminados a cada grupo de interés usando SparkR. Este tutorial se enfoca en la programación de SparkR y no en el análisis del problema. Para abordar este tutorial, el lector debe tener suficiencia en los módulos correspondientes de analítica predictiva.

## Definición del problema

Un vendedor desea enviar publicidad electrónica a una población de adolecentes y adultos jóvenes con el fin de maximizar sus ventas. Para ello, desea poder clasificar a sus clientes potenciales por grupos de interés de acuerdo con sus intereses y consecuentemente enviar publicidad específica a cada uno de ellos.   

En este problema se desea determina que grupos de interés existen en una población de clientes a partir de los mensajes enviados por un servicio de redes sociales. La información disponible consiste en 30000 observaciones de 40 variables que podrían caracterizar los intereses de la población analizada. Estas variables corresponden a palabras que pueden asociarse a un interés de la poblaión analizada. Cada variable mide la frecuencia con que una determinada palabra aparece en los mensajes de texto; adicionalmente, dentro de estas variables se incluye  información como el sexo, la edad y la cantidad de contactos de la persona. 

## Solución

### Preparación

In [1]:
##
## Esta función se usará para ejecutar comandos en el 
## sistema operativo y capturar la salida.
##
systemp <- function(command) cat(system(command, intern = TRUE), sep = '\n')

In [2]:
##
## Se procede a la carga de la librería
##
library(SparkR)
sparkR.session(enableHiveSupport = TRUE)


Attaching package: ‘SparkR’

The following objects are masked from ‘package:stats’:

    cov, filter, lag, na.omit, predict, sd, var, window

The following objects are masked from ‘package:base’:

    as.data.frame, colnames, colnames<-, drop, endsWith, intersect,
    rank, rbind, sample, startsWith, subset, summary, transform, union

Spark package found in SPARK_HOME: /usr/local/spark


Launching java with spark-submit command /usr/local/spark/bin/spark-submit   sparkr-shell /tmp/RtmpcPFt5S/backend_port1a8066ac96d4 


Java ref type org.apache.spark.sql.SparkSession id 1 

### Carga de datos

El archivo con los datos se encuentra en la carpeta actual de trabajo en la máquina local.

In [3]:
## copia el archivo al HDFS
systemp('hdfs dfs -copyFromLocal snsdata.csv /tmp/snsdata.csv') 

Warning message in system(command, intern = TRUE):
“running command 'hdfs dfs -copyFromLocal snsdata.csv /tmp/snsdata.csv' had status 1”

In [4]:
df <- read.df(
    '/tmp/snsdata.csv',    # ubicación y nombre del archivo
    'csv',                 # formato
    header = TRUE)         # encabeamiento

head(df)

gradyear,gender,age,friends,basketball,football,soccer,softball,volleyball,swimming,...,blonde,mall,shopping,clothes,hollister,abercrombie,die,death,drunk,drugs
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,...,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2006,M,18.982,7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2006,F,18.801,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2006,M,18.335,69,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2006,F,18.875,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2006,NA,18.995,10,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,1,1
2006,F,NA,142,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [5]:
##
## Cantidad de registros leidos
##
count(df)

[1] 30000

In [6]:
##
## Tipos de datos de los campos del DataFrame
##
printSchema(df)

root
 |-- gradyear: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- friends: string (nullable = true)
 |-- basketball: string (nullable = true)
 |-- football: string (nullable = true)
 |-- soccer: string (nullable = true)
 |-- softball: string (nullable = true)
 |-- volleyball: string (nullable = true)
 |-- swimming: string (nullable = true)
 |-- cheerleading: string (nullable = true)
 |-- baseball: string (nullable = true)
 |-- tennis: string (nullable = true)
 |-- sports: string (nullable = true)
 |-- cute: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- sexy: string (nullable = true)
 |-- hot: string (nullable = true)
 |-- kissed: string (nullable = true)
 |-- dance: string (nullable = true)
 |-- band: string (nullable = true)
 |-- marching: string (nullable = true)
 |-- music: string (nullable = true)
 |-- rock: string (nullable = true)
 |-- god: string (nullable = true)
 |-- church: string (nullable = true)
 |

### Análisis exploratorio

A continuación se ejemplifican algunos cómputos típicos para el análisis exploratorio.

In [7]:
##
## Conteo por género
##
collect(
    summarize(
        groupBy(df,          ## DataFrame
                df$gender),  ## Columna para realizar la agregación
        count=n(df$gender))) ## Cuenta la cantidad de registros

gender,count
<chr>,<dbl>
F,22054
NA,0
M,5222


In [8]:
##
## Conteo por género
##
table(collect(select(df, 'gender')))


    F     M 
22054  5222 

In [9]:
##
## Estadísticos por rango de edades
##
collect(summary(select(df, 'age')))

summary,age
<chr>,<chr>
count,24914
mean,17.993949546439772
stddev,7.858054477853863
min,10.842
25%,16.309
50%,17.287
75%,18.259
max,98.708


Los métodos que pueden aplicarse a las columnas están listados en: https://spark.apache.org/docs/latest/api/R/column_nonaggregate_functions.html

In [10]:
##
## Cantidad de nulos en la columna age
##
count(filter(df, isNull(df$age)))

[1] 5086

In [11]:
##
## Se agrega una columna con las edades entre 13 y 19,
## reemplazando por null los valores por fuera de este
## rango
##
df <- withColumn(
    df, 
    'age1319',
    otherwise(when((df$age >= 13) & (df$age < 20), 
                   df$age),
             lit(NaN)))

##
## Se verifican los valores en la nueva columna
##
collect(describe(select(df, 'age1319')))

summary,age1319
<chr>,<chr>
count,24477
mean,17.25242893328433
stddev,1.1574649278955391
min,13.027
max,19.995


In [12]:
##
## Se calcula la edad promedio por año 
## de graduación para la muestra en el
## rango de edades considerado
##

age_df <- 
summarize(
    groupBy(df,             ## DataFrame
            df$gradyear),   ## Columna para realizar la agregación
    avgage=avg(df$age1319)) ## Cuenta la cantidad de registros por valor en quantity

collect(age_df)

gradyear,avgage
<chr>,<dbl>
2009,15.81957
2006,18.65586
2008,16.76770
2007,17.70617


### Entrenamiento del modelo

In [13]:
model <- spark.kmeans(
    df,
    ~ friends + basketball + football + soccer + softball + 
      volleyball + swimming + cheerleading + baseball + tennis + 
      sports + cute + sex + sexy + hot + kissed + dance + band + 
      marching + music + rock + god + church + jesus + bible + 
      hair + dress + blonde + mall + shopping + clothes + hollister + 
      abercrombie + die + death + drunk + drugs,
    k = 5
)

## summary(model)

In [14]:
##
## Pronostica a que cluster pertenece cada patrón
##
df <- predict(model, df)

### Análisis del modelo

In [15]:
collect(count(groupBy(df, 'prediction')))

prediction,count
<int>,<dbl>
1,3563
3,4837
4,7141
2,2587
0,11872


In [16]:
##
## Características demográficas de los clusters.
## Edad por cluster.
##
collect(
    summarize(
        groupBy(df,             ## DataFrame
                df$prediction), ## Columna para realizar la agregación
        ave_age_by_cluster=avg(df$age1319)))

prediction,ave_age_by_cluster
<int>,<dbl>
1,17.10497
3,17.22490
4,17.32426
2,17.02875
0,17.31829
